<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Train_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/uc?id=1dxd2pySfCIDJYG7qMtuJre8ph068xc1X

In [ ]:
# !gdown https://drive.google.com/uc?id=1sYR9EgHkM0oiGRQVlFQCyHO8kMRJ4ibQ

In [ ]:
# !gdown https://drive.google.com/uc?id=1X4cmMYRjxXFomCJ1adMhPNMYtd4WeDHP

In [ ]:
# !gdown https://drive.google.com/uc?id=1yAeNEPXD3LNxmtSnC09jEg0vEDc1qiaq

In [ ]:
# !unzip test.zip

In [ ]:
# !unzip train.zip

# Training Dataset Preparation

In [ ]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = "/content/Dataset(s)/mm-imdb/fused/train"
labels_file = '/content/Dataset(s)/mm-imdb/fused/train_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [ ]:
len(image_file_paths)

46656

In [ ]:
# genre_labels

In [ ]:
# unique_labels_set = set()

# for labels in genre_labels:
#     unique_labels_set.update(labels)

# unique_labels = sorted(list(unique_labels_set))

In [ ]:
# unique_labels

In [ ]:
# len(unique_labels)

In [ ]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [ ]:
# Convert the defaultdict into a list of tuples
label_count_list = [(label, count) for label, count in label_counts.items()]

# Sort the list of tuples based on counts in descending order
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)

# Display the sorted list
for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

Drama: 25272
Comedy: 15324
Romance: 9678
Thriller: 9339
Crime: 6879
Action: 6465
Adventure: 4833
Horror: 4809
Documentary: 3702
Mystery: 3693
Sci-Fi: 3636
Fantasy: 3486
Family: 2934
War: 2418
Biography: 2364
History: 2040
Music: 1902
Animation: 1758
Musical: 1509
Western: 1269
Sport: 1137
Short: 843
Film-Noir: 606
News: 117
Talk-Show: 6
Reality-TV: 3
Total Labels:  26


In [ ]:
min_label_count = 500
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [ ]:
valid_labels, len(valid_labels)

(['Action',
  'Adventure',
  'Animation',
  'Biography',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Family',
  'Fantasy',
  'Film-Noir',
  'History',
  'Horror',
  'Music',
  'Musical',
  'Mystery',
  'Romance',
  'Sci-Fi',
  'Short',
  'Sport',
  'Thriller',
  'War',
  'Western'],
 23)

In [ ]:
# filtered_image_paths = []
# filtered_genre_labels = []

# for image_path, labels in zip(image_file_paths, genre_labels):
#     valid_labels_for_sample = [label for label in labels if label in valid_labels]

#     if valid_labels_for_sample:
#         filtered_image_paths.append(image_path)
#         filtered_genre_labels.append(valid_labels_for_sample)

In [ ]:
# len(filtered_image_paths)

In [ ]:
# len(filtered_genre_labels)

In [ ]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    multi_hot_labels.append(multi_hot)

In [ ]:
multi_hot_labels[1]

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [ ]:
genre_labels[1]

['Crime', 'Drama', 'Thriller']

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create a custom dataset
train_dataset = CustomMultiLabelDataset(image_file_paths, multi_hot_labels, transform=transform)

In [ ]:
len(train_dataset)

46656

In [ ]:
train_dataset[10]

(tensor([[[-0.4739, -1.2103, -1.2617,  ..., -1.2617, -1.2103, -0.4568],
          [-0.7137, -2.0494, -2.0837,  ..., -2.0665, -2.0323, -0.7137],
          [-0.6281, -2.1179, -2.1179,  ..., -2.1179, -2.1008, -0.6281],
          ...,
          [ 0.5193,  1.7865,  1.7352,  ...,  1.9920,  2.2489,  0.6221],
          [ 0.5022,  1.7865,  1.7009,  ...,  2.0263,  2.2318,  0.6563],
          [ 0.2624,  1.0159,  1.0159,  ...,  1.1015,  1.2899,  0.3138]],
 
         [[-0.3550, -1.1078, -1.1604,  ..., -1.1604, -1.1078, -0.3375],
          [-0.6001, -1.9657, -2.0007,  ..., -1.9832, -1.9482, -0.6001],
          [-0.5126, -2.0357, -2.0357,  ..., -2.0357, -2.0182, -0.5126],
          ...,
          [ 0.6954,  2.0434,  2.0609,  ...,  2.1134,  2.3936,  0.6954],
          [ 0.8004,  2.1660,  2.0959,  ...,  2.2010,  2.4111,  0.7654],
          [ 0.4503,  1.2556,  1.3081,  ...,  1.3256,  1.4657,  0.4678]],
 
         [[-0.1312, -0.8807, -0.9330,  ..., -0.9330, -0.8807, -0.1138],
          [-0.3753, -1.7347,

# Test Dataset Preparation

In [ ]:
import os

test_image_file_paths = []
test_genre_labels = []

image_folder_add = "/content/Dataset(s)/mm-imdb/fused/test"
labels_file = '/content/Dataset(s)/mm-imdb/fused/test_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces

        if not (filename.endswith("_1.png") or filename.endswith("_2.png")):
            image_path = os.path.join(image_folder_add, filename)
            test_image_file_paths.append(image_path)
            test_genre_labels.append(labels)


In [ ]:
len(test_image_file_paths)

7799

In [ ]:
test_multi_hot_labels = []

for labels in test_genre_labels:
    test_multi_hot = [1 if label in labels else 0 for label in valid_labels]
    test_multi_hot_labels.append(test_multi_hot)

In [ ]:
test_dataset = CustomMultiLabelDataset(test_image_file_paths, test_multi_hot_labels, transform=transform)

In [ ]:
len(test_dataset)

7799

# Training

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = models.resnet101(pretrained=True)  # You can use a pre-trained model

# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 512),
#     nn.ReLU(),
#     nn.Linear(512, len(unique_labels)),
#     nn.Sigmoid()
# )

model = models.resnet101(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(valid_labels))
model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

In [25]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    # if epoch > 10:
    save_dir = "/content/Model/Models-Train-21/"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    model_name = str(epoch) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(model.state_dict(), save_path)

    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            # loss = criterion(outputs, labels.float())

            # Apply a threshold (e.g., 0.5) to convert logits to binary predictions
            predictions.extend((outputs > 0.5).int().cpu().numpy())
            true_labels.extend(labels.int().cpu().numpy())

    # Calculate evaluation metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(classification_report(true_labels, predictions))

Epoch 1/30, Loss: 0.2674324148333285


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.09488395948198487
Precision: 0.4177244738685913
Recall: 0.07366235027532383
F1-Score: 0.10502846437252289
              precision    recall  f1-score   support

           0       0.64      0.04      0.07      1044
           1       0.64      0.01      0.02       821
           2       1.00      0.04      0.07       306
           3       0.00      0.00      0.00       411
           4       0.74      0.16      0.27      2611
           5       0.73      0.18      0.29      1163
           6       0.30      0.01      0.02       629
           7       0.68      0.64      0.66      4142
           8       0.80      0.05      0.10       518
           9       0.00      0.00      0.00       585
          10       0.00      0.00      0.00       102
          11       0.33      0.01      0.01       345
          12       0.84      0.13      0.23       825
          13       0.00      0.00      0.00       311
          14       0.00      0.00      0.00       253
          15     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.1224515963585075
Precision: 0.5767241273348941
Recall: 0.17661950780315613
F1-Score: 0.22794248626308267
              precision    recall  f1-score   support

           0       0.66      0.25      0.36      1044
           1       0.70      0.02      0.04       821
           2       0.88      0.09      0.17       306
           3       0.60      0.01      0.01       411
           4       0.75      0.27      0.40      2611
           5       0.74      0.28      0.41      1163
           6       0.84      0.41      0.55       629
           7       0.78      0.57      0.66      4142
           8       0.89      0.09      0.16       518
           9       1.00      0.00      0.00       585
          10       0.00      0.00      0.00       102
          11       0.56      0.01      0.03       345
          12       0.75      0.43      0.55       825
          13       0.33      0.00      0.01       311
          14       0.00      0.00      0.00       253
          15      

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 29/30, Loss: 0.017000379968327258
Accuracy: 0.12809334530067956
Precision: 0.5447906118471988
Recall: 0.36925890262597844
F1-Score: 0.4261342067638938
              precision    recall  f1-score   support

           0       0.61      0.40      0.48      1044
           1       0.61      0.22      0.32       821
           2       0.52      0.40      0.45       306
           3       0.26      0.21      0.23       411
           4       0.67      0.51      0.58      2611
           5       0.63      0.44      0.52      1163
           6       0.68      0.66      0.67       629
           7       0.67      0.79      0.72      4142
           8       0.72      0.24      0.35       518
           9       0.50      0.18      0.26       585
          10       0.14      0.03      0.05       102
          11       0.30      0.27      0.28       345
          12       0.78      0.44      0.56       825
          13       0.46      0.44      0.45       311
          14       0.43      0.1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 30/30, Loss: 0.015807005217163446
Accuracy: 0.12283626105911014
Precision: 0.5401968844635666
Recall: 0.3813951100941434
F1-Score: 0.4320822426196188
              precision    recall  f1-score   support

           0       0.59      0.44      0.50      1044
           1       0.50      0.38      0.43       821
           2       0.57      0.32      0.41       306
           3       0.34      0.09      0.15       411
           4       0.61      0.62      0.62      2611
           5       0.51      0.62      0.56      1163
           6       0.79      0.53      0.63       629
           7       0.70      0.73      0.71      4142
           8       0.62      0.30      0.40       518
           9       0.44      0.25      0.32       585
          10       0.18      0.12      0.14       102
          11       0.38      0.23      0.29       345
          12       0.81      0.37      0.51       825
          13       0.58      0.23      0.32       311
          14       0.36      0.20

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# New Section

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

save_dir = "/content/Model/Models-Train-1"
load_path = os.path.join(save_dir, '8_model.pth')


model = models.resnet101(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, len(unique_labels)),
    nn.Sigmoid()
)
model.load_state_dict(torch.load(load_path))
model.eval()
model.to(device)

In [ ]:
# Initialize lists to store predicted and ground truth labels
predicted_labels = []
true_labels = []

# Iterate through the test set and make predictions
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted_labels.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
predicted_labels[0]

In [ ]:
threshold = 0.5

binary_predictions = []

for sample_logits in predicted_labels:
    binary_sample_predictions = [1 if value >= threshold else 0 for value in sample_logits]
    binary_predictions.append(binary_sample_predictions)

In [ ]:
binary_predictions[1000]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(true_labels, binary_predictions)
precision = precision_score(true_labels, binary_predictions, average='macro')
recall = recall_score(true_labels, binary_predictions, average='macro')
f1 = f1_score(true_labels, binary_predictions, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(classification_report(true_labels, binary_predictions))